In [2]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Current iam_role is None
iam_role has been set to arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role.


In [4]:
%region us-west-2

Previous region: None
Setting new region to: us-west-2
Region is set to: us-west-2


In [0]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Additional python modules to be included:
s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Previous number of workers: 5
Setting new number of workers to: 2
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: c7ff10bc-1e92-4e3c-b291-349c9b5897bf
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
--additional-python-modules s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Waiting for session c7ff10bc-1e92-4e3c-b291-349c9b5897bf to get into ready status...
Session c7ff10bc-1e92-4e3c-b291-349c9b5897bf has been created




In [1]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [2]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [3]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [4]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [5]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [6]:
from sstm_transformation.tsm_builder import TSMBuilder

In [7]:
#Read All dfs 
df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")
df_contact =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/")
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")
df_meds = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [8]:
builder = TSMBuilder("sstm.yaml", spark=spark)


In [9]:
#Upload data on S3 
def upload_result(result):
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")


In [10]:
#PeopleType Transformation
def people_type_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peopletypes(contact_df=df_contact)
    print(result)
    upload_result(result)
    
people_type_transformation(df_contact)

{'PeopleTypes': DataFrame[People_Type: string, Truve_Org_ID: int, Client_Org_ID: int, Custom1: string, Custom2: string, Custom3: string, People_Sub_Type: string, People_Type_Id: bigint]}
Writing PeopleTypes


In [11]:
#Peoples Transformation
def people_master_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peoplemaster(contact_df=df_contact)
    print(result)
    upload_result(result)
    
people_master_transformation(df_contact)

root
 |-- People_ID: string (nullable = true)
 |-- People_Type: string (nullable = true)

root
 |-- People_Type: string (nullable = true)
 |-- Truve_Org_ID: integer (nullable = true)
 |-- Client_Org_ID: integer (nullable = true)
 |-- Custom1: string (nullable = true)
 |-- Custom2: string (nullable = true)
 |-- Custom3: string (nullable = true)
 |-- People_Sub_Type: string (nullable = true)
 |-- People_Type_Id: long (nullable = true)

{'Peoples': DataFrame[Truve_Org_ID: int, Client_Org_ID: int, People_ID: string, First_Name: string, Middle_Name: string, Last_Name: string, Date_of_Birth: string, Gender: string, Custom1: string, Custom2: string, Custom3: string], 'Peoples_PeopleType': DataFrame[People_ID: string, People_Type_ID: bigint]}
Writing Peoples
Writing Peoples_PeopleType


In [12]:
#Peoples Transformation
def projecttype_transformation(df_projecttype):
    #Read Raw Data
    result = builder.build_practicetypes(df_projecttype)
    print(result)
    upload_result(result)
    
projecttype_transformation(df_projecttype)

root
 |-- Truve_Org_ID: integer (nullable = false)
 |-- Client_Org_ID: integer (nullable = false)
 |-- Practice_Type_ID: long (nullable = true)
 |-- Practice_Type_Name: string (nullable = true)
 |-- Custom1: string (nullable = true)
 |-- Custom2: string (nullable = true)
 |-- Custom3: string (nullable = true)

{'PracticeTypes': DataFrame[Truve_Org_ID: int, Client_Org_ID: int, Practice_Type_ID: bigint, Practice_Type_Name: string, Custom1: string, Custom2: string, Custom3: string]}
Writing PracticeTypes


In [13]:
#Peoples Transformation
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")

def phases_transformation(df_projecttype):
    #Read Raw Data
    
    result = builder.build_phasemaster(df_projecttype)
    print(result)
    upload_result(result)
    
phases_transformation(df_projecttype)

+------------+-------------+--------+--------------------+----------------+-----------+--------------+------------------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Phase_ID|          Phase_Name|Practice_Type_ID|Phase_Order|Phase_Category|Phase_Sub_Category|Custom1|Custom2|Custom3|
+------------+-------------+--------+--------------------+----------------+-----------+--------------+------------------+-------+-------+-------+
|        6586|         6586|   91388|            New Lead|           18764|       null|          null|              null|   null|   null|   null|
|        6586|         6586|  103016|              Intake|           18764|       null|          null|              null|   null|   null|   null|
|        6586|         6586|   91389|         File Set-Up|           18764|       null|          null|              null|   null|   null|   null|
|        6586|         6586|   91390|           Treatment|           18764|       null|          null|              null|   

In [14]:
#Peoples Transformation
def value_transformation(df_meds):
    #Read Raw Data
    result = builder.build_casefigures(df_meds)
    print(result)
    upload_result(result)
    
value_transformation(df_meds)

+------------+-------------+--------------+--------+--------------+-------------------+-----------+-------------+--------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Case_Figure_ID|        Figure_Type|Figure_Date|Figure_Status|   Value|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+--------------+-------------------+-----------+-------------+--------+-------+-------+-------+
|        6586|         6586|       9675645| 9675645|          null|      Case Expenses|       null|         null|    20.0|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|          null|      Case Expenses|       null|         null|   72.88|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|          null|      Case Expenses|       null|         null|   72.88|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|          null|      Case Expenses|       null|         null|    3

In [15]:
#Peoples Transformation
def cases_transformation(df_project):
    #Read Raw Data
    result = builder.build_casemaster(df_project)
    print(result)
    upload_result(result)
    
cases_transformation(df_project)

root
 |-- Truve_Org_ID: integer (nullable = false)
 |-- Client_Org_ID: integer (nullable = false)
 |-- Case_ID: integer (nullable = true)
 |-- Practice_Type_ID: string (nullable = true)
 |-- Is_Archived: string (nullable = true)
 |-- Incident_Date: string (nullable = true)

{'Cases': DataFrame[Truve_Org_ID: int, Client_Org_ID: int, Case_ID: int, Practice_Type_ID: string, Is_Archived: string, Incident_Date: string]}
Writing Cases


In [16]:
#Peoples Transformation
def intake_transformation(df_intake):
    #Read Raw Data
    result = builder.build_intakesummary(df_intake)
    return result 

result = intake_transformation(df_intake)
result["IntakeDetails"] = result["IntakeDetails"].drop("referral_fee")
upload_result(result)


+------------+-------------+--------------+--------+---------+---------------------------+-------------+--------------+----------------+----------------+------------+------------+-------------------+-----------------+-----------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Intake_ID|Person_Performing_Intake_ID|Intake_Source|Date_of_Intake|Date_of_Incident|DUI_or_HitandRun|Referral_Fee|Referral_Fee|If_Case_Referred_In|If_Qualified_Case|If_VIP_Lead|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+---------+---------------------------+-------------+--------------+----------------+----------------+------------+------------+-------------------+-----------------+-----------+-------+-------+-------+
|        6586|         6586|       9252629| 9252629|  9252629|                   30625608|         null|    2021-04-28|      2021-04-22|            None|        null|        null|               null|             null|       null|   null| 

In [17]:
#Peoples Transformation
def casesummary_transformation(df_casesummary):
    #Read Raw Data
    result = builder.build_casesummary(df_casesummary)
    print(result)
    upload_result(result)
    
casesummary_transformation(df_casesummary)

+------------+-------------+--------------+--------+------------+----------------+----------------+---------+-------------------+-----------+--------------------------+-----------------------+--------------+------------+------------+------------+--------------+---------------------+----------------+-----------------------+-------------------------+---------------------------+----------------------+------------------------+-----------------------+-----------+--------------------+-------------+-------+-------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Case_Type_ID|Case_Create_Date|Date_of_Incident|Case_Name|Plaintiff_Full_Name|Attorney_ID|Prelitigation_Paralegal_ID|Litigation_Paralegal_ID|CaseManager_ID|Cocounsel_ID|Case_Team_ID|Insurance_ID|Case_Status_ID|Case_Marketing_Source|Case_Source_Name|Attorney_Fee_Percentage|Projected_Settlement_Date|Projected_Settlement_Amount|Actual_Settlement_Date|Actual_Settlement_Amount|If_Case_Settled_Presuit|If_VIP_Case|

In [ ]:
builder._get_table_config("Phases")
sstm_config

In [ ]:
def people_master_transformation():
    #Read Raw Data
    contact_dyf = glueContext.create_dynamic_frame_from_options(\
    connection_type = "s3", \
    connection_options = {
        "paths": ["s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/historical_contacts.parquet"]}, \
    format = "parquet",
    )
    contact_sp_df = contact_dyf.toDF()
    print(contact_sp_df.count())
    sstm_peoplemaster_df = builder.build_peoplemaster(contact_df=contact_sp_df)
    trans_obj = DynamicFrame.fromDF(sstm_peoplemaster_df, glueContext, "sstm_peoplemaster")
    print(trans_obj)
    #sstm_peoplemaster_df.write.mode("overwrite").format("parquet").save("s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/peoplemaster/")
    #print("PeopleTypes Processed successfully")

In [ ]:
#people_type_transformation()
people_master_transformation()

In [ ]:
df_forms =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [ ]:
df_forms.count()

In [ ]:
df_forms.printSchema()

In [ ]:
df_forms.show()

In [ ]:
from awsglue.dynamicframe import DynamicFrame

dyf_casesummary = DynamicFrame.fromDF(df_forms, glueContext, "nested")


In [ ]:
glueContext.write_dynamic_frame.from_options(frame = dyf_casesummary,
                                         connection_type = "s3",
                                         connection_options = {"path": "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/casesummary/"},
                                         format = "parquet")

In [ ]:
writeToFile(df_forms,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/casesummary/", "overwrite")


In [8]:
%stop_session

Stopping session: c7ff10bc-1e92-4e3c-b291-349c9b5897bf
Stopped session.


In [ ]:
df_forms =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collection/meds.parquet")


In [ ]:
df_forms.count()

In [ ]:
df_forms.printSchema()

In [ ]:
df_forms.show(n=10)

In [ ]:
writeToFile(df_forms,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/meds/", "overwrite")


In [ ]:
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")


In [ ]:
df_intake.count()

In [ ]:
df_intake.show(n=10)

In [ ]:
df_intake.printSchema()

In [ ]:
writeToFile(df_intake,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/intake/", "overwrite")


In [ ]:
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [ ]:
df_casesummary.count()

In [ ]:
df_casesummary.show()

In [ ]:
def write_parquet_files(df, name):
    df_rept = df.repartition(1)
    writeToFile(df_rept,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/{}/".format(name), "overwrite")


In [ ]:
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")


In [ ]:
write_parquet_files(df_intake, "intake")

In [ ]:
df_intake.count()

In [ ]:
df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
write_parquet_files(df_project, "project")

In [ ]:
df_meds =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
write_parquet_files(df_meds, "meds")